In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.gaussian_process import GaussianProcess

In [33]:
#Get train, test datas 
from tools.data import get_datasets
train, test = get_datasets()
print train.shape
print test.shape

Reading data file
(30000, 2050)
(10983, 2050)


In [34]:
# binary classification: Noise vs Digit
# Digit -> 1
# Noise -> 0
#Reducing train,test sets to use the GPC
train = train[:(len(train)/3),:]
test = test[:(len(test)/3), :]

train[train[:,-1] >= 0,-1] =  0
test[test[:,-1] >= 0,-1] =  0

train[:,-1] = train[:,-1] + 1
test[:,-1] = test[:,-1] + 1

train_target = train[:,-1]
test_target = test[:,-1]

DECISIONBOUNDARY = 0

In [35]:
#FastFourierTransformation apply to the sets using the 4 sensors

train_f = np.zeros(train.shape)

for i in range(len(train_f)):
    length = train[i,0]
    train_f[i,0] = train[i,0]
    train_f[i,-1] = train[i,-1]
    
    for j in range(4):
        train_f[i, 1+512*j:1+512*j+length] = np.abs(np.fft.fft(train[i, 1+512*j:1+512*j+length]))
        
test_f = np.zeros(test.shape)

for i in range(len(test_f)):
    length = test[i,0]
    test_f[i,0] = test[i,0]
    test_f[i,-1] = test[i,-1]
    
    for j in range(4):
        test_f[i, 1+512*j:1+512*j+length] = np.abs(np.fft.fft(test[i, 1+512*j:1+512*j+length]))

In [36]:
#PCA with 25 components
pca = PCA(n_components = 25)
train_principal = pca.fit_transform(train_f)
test_principal = pca.transform(test_f)

In [ ]:
#Fit a gaussian process model to the data
#gp = GaussianProcess(theta0=10000e-1, random_start=1000)
#gp.fit(train_principal, train_target)
#Generate a set of predictions for the test data
#y_pred = gp.predict(test_principal)
#Convert the continuous predictions into the classes
#by splitting on a decision boundary of 0
#predictions = []
#for y in y_pred:
    #if y > DECISIONBOUNDARY:
            #predictions.append(1)
    #else:
            #predictions.append(0)
#print "----------------"
#print "Classification Report:"
#print metrics.classification_report(test_target, predictions)
#Produce results around 75% 

In [1]:
#erreur = []
#total_correct = 0
#for i in range(test_principal.shape[0]):
#    if np.array(predictions)[i] == test_target[i]:
#        total_correct += 1

#erreur.append(float(total_correct) / test_principal.shape[0] * 100)
#print 'Pourcentage correct %f:' % float(total_correct) / test_principal.shape[0] * 100
#print "Pourcentage correct: %d" %np.max(erreur)

In [15]:
#K-PPV algorithm for several values of K
erreur = []
x = np.arange(1,20,1)
for n in (x):
    # train set
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_principal, train[:,-1])

    #test set
    total_correct = 0
    for i in range(test_principal.shape[0]):
        if neigh.predict(test_principal[i])[0] == test[i,-1]:
            total_correct += 1

    erreur.append(float(total_correct) / test_principal.shape[0] * 100)
    print 'Pourcentage correct %d:' % n, float(total_correct) / test_principal.shape[0] * 100
    


Pourcentage correct 1: 98.8709824274
Pourcentage correct 2: 98.8800874078
Pourcentage correct 3: 98.798142584
Pourcentage correct 4: 98.8436674861
Pourcentage correct 5: 98.7799326231
Pourcentage correct 6: 98.7617226623
Pourcentage correct 7: 98.7435127014
Pourcentage correct 8: 98.6797778385
Pourcentage correct 9: 98.5887280342
Pourcentage correct 10: 98.5796230538
Pourcentage correct 11: 98.5249931713
Pourcentage correct 12: 98.5978330147
Pourcentage correct 13: 98.4703632887
Pourcentage correct 14: 98.5067832104
Pourcentage correct 15: 98.4430483474
Pourcentage correct 16: 98.49767823
Pourcentage correct 17: 98.3884184649
Pourcentage correct 18: 98.3884184649
Pourcentage correct 19: 98.3246836019


In [16]:
print "le meilleur pourcentage : %.3f "% np.max(erreur)
print "avec k voisins = %d" % x[np.argmax(erreur)]

le meilleur pourcentage : 98.880 
avec k voisins = 2
